In [1]:
import glob
import pandas as pd
import numpy as np
import os, sys
import collections
import math
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier 

In [3]:
from numba.typed import List
from numba import jit, njit, vectorize

In [4]:
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

In [5]:
sample_rate = 50  # number of observation per second based on dataset documentation(150 samples in 3 second)

sliding_size = int((1/3) * sample_rate)  # number of skipped datapoints to start next window
print(sliding_size)

16


# Feature sets

In [6]:
@njit()
def mean_crossing_rate(col):
    # col = np.array(values)
    normalized = col - col.mean()  # to make elements of array possitive or negetive
    return ((normalized[:-1] * col[1:]) < 0).sum()  # Zero-Crossing_rate

@njit()
def iqr(window):  # inter-quartile range
    Q1 = np.median(window[:len(window)//2])  # First quartile (Q1) 
    Q3 = np.median(window[len(window)//2:])  # Third quartile (Q3) 
    IQR = Q3 - Q1 # Interquartile range (IQR) 
    return(IQR) 
@njit()
def calc_sma_for_window(data):
    return np.sum(data) / len(data)  
@njit()
def get_min(x):
    m = np.min(x)
    return m
@njit()
def get_max(x):
    m = np.max(x)
    return m
@njit()
def get_mean(x):
    m = np.mean(x)
    return m
@njit()
def get_var(x):
    m = np.var(x)
    return m
@njit()
def get_sum(x):
    m = x.sum()
    return m 
@njit()
def get_median(x):
    m = np.median(x)
    return m 
@njit()
def get_std(x):
    m = np.median(x)
    return m 
@njit()
def get_rng(x):
    n = np.max(x)
    m = np.min(x)
    z = n-m
    return z 

def get_entropy(Y):
    """
    Also known as Shanon Entropy
    Reference: https://en.wikipedia.org/wiki/Entropy_(information_theory)
    """
    unique, count = np.unique(Y, return_counts=True, axis=0)
    prob = count/len(Y)
    en = np.sum((-1)*prob*np.log2(prob))
    return en

In [7]:
def Energy(frame):
    return sum( [ abs(x)**2 for x in frame ] ) / len(frame)

# Feature extraction and Vector_Normalization on signal

In [8]:
def Features(window):# mean, std,max,min and zero-crossing-rate
    wind = window.iloc[:, :-1]
    win = np.array(wind)
    
    data_normalized = preprocessing.normalize(win, norm  = 'l1') # normalization on signal
    win = data_normalized
    features = []
    
    features.append(get_mean(win))
    features.append(get_median(win))
    features.append(get_std(win))
    features.append(get_var(win))
    features.append(get_min(win))
    features.append(get_max(win))
    features.append(get_sum(win))
    features.append(calc_sma_for_window(win))
    features.append(get_rng(win))
    features.append(get_entropy(win))
    mean_crossing = [mean_crossing_rate(win[:, i]) for i in range(win.shape[1])]
    features.append(np.array(mean_crossing))
    IQR = iqr(win)
    features.append(np.array(IQR))
    energy_measure = Energy(win)
    features.append(np.array(energy_measure))
 

    
    features = np.hstack(features).tolist()
    
    label = window.iloc[:, -1].mode()[0]  ## select the most frequent label as the label of the window
    features.append(label)
    return features

In [9]:
def windowing_dataset(dataset, win_size, feature_extraction_function, subject_id, overlap=False):
    windowed_dataset = []
    win_count = 0
    if overlap:
        step_size = sliding_size  # for Overlapping technique
    else:
        step_size = win_size  # for Non-overlapping technique

    for index in range(0, dataset.shape[0], step_size):
        start = index
        end = start + win_size
        # to assure all of windows are equal in size
        if (end <= dataset.shape[0]):
            window = dataset.iloc[start:end, :].reset_index(drop=True)
            win_count = win_count + 1
            features = feature_extraction_function(window)

            windowed_dataset.append(features)

    final = pd.DataFrame(windowed_dataset)
    final.insert(0, 'group', subject_id)  # to use in Subject CV
    return final

In [10]:
def Preprocessing(dataset_path, overlapping):
    feature_function = Features
    win_size = 3

    print("Start for win size {}".format(win_size))
    datapoints_per_window = int(win_size * sample_rate)

    print(feature_function.__name__)

    ALL = []
    List = [2,5,15]
    for subject in List:
        file_path = dataset_path + '\subject{0}_mutual6.csv'.format(subject)
        all_cols = []
        
        
#########################################################################################################################
        for i in range(2, 117, 13):# indices of accelarations
            indices = list(range(i, i + 13))
            all_cols.extend(indices)

        all_cols.append(119)  # label index

        tmp_db = pd.read_csv(file_path, header=None, usecols=all_cols, sep='\t')
        tmp_db.columns = list(range(tmp_db.shape[1]))  # re-index the columns

        transformed_db = windowing_dataset(tmp_db, datapoints_per_window, feature_function, subject,
                                                   overlap=overlapping)

        ALL.append(transformed_db)
#########################################################################################################################
       
    #final_dataset = pd.DataFrame()
    ALL_dataset = pd.DataFrame()
    ALL_dataset = ALL_dataset.append(ALL, ignore_index=True)
    
    return ALL_dataset
   

In [11]:
def subject_cross_validation(X, Y, groups, classifier):
    f1 = []
    logo = LeaveOneGroupOut()
    i = 0
    for train_index, test_index in logo.split(X, Y, groups=groups):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]
        i += 1
        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)
        f = f1_score(y_true=y_test, y_pred=y_pred, average='micro')
        
        print('Model-',i ,' -',' f1 score: ', f)
       
        f1.append(f)
    return np.mean(f1)

In [12]:
def classifier(dataset, model):
    results = dict()
    win_size = float(3)
    print('window_size = ', win_size,' sec')

    dataset = dataset
    groups = dataset['group']
    X = dataset.iloc[:, 1:-1]
    X = np.array(X)
    
    Y = dataset.iloc[:, -1]
    Y = np.array(Y)

    for model_name, mod in model.items():
        f1 = 0

        f1 = subject_cross_validation(X, Y, groups, mod)

        if win_size in results:
            results[win_size].append(f1)
        else:
            results[win_size] = [f1]


        results = collections.OrderedDict(sorted(results.items()))

        final = []
        col = list(model.keys())
        col.insert(0, "window-size")
        final.append(col)
        for k, v in results.items():
            tmp = []
            tmp.append([k])
            tmp.append(v)
            flattened = [val for sublist in tmp for val in sublist]
            final.append(flattened)

    accuracy = final[1][1]
    
    return accuracy

In [13]:
mod = {'RF': RandomForestClassifier(n_estimators=40, random_state=42, n_jobs=-1)}

In [14]:
dataset_csv_path = r"D:\projec\proj\data_mutual6"

overlapping = 1  # input 0 for non overlapping, 1 for overlapping

ALL = Preprocessing(dataset_path=dataset_csv_path, overlapping=bool(int(overlapping)))

Start for win size 3
Features


In [15]:
ALL.head(10)

,group,0,1,2,3,4,5,6,7,8,...,236,237,238,239,240,241,242,243,244,245
0,2,-0.000183,0.000106,0.000106,0.000333,-0.075511,0.085885,-3.205252,-0.021368,0.161396,...,1.614153e-06,0.000009,0.000030,1.164846e-06,1.635241e-06,0.000010,0.000009,0.000017,0.000008,0
1,2,-0.000211,0.000116,0.000116,0.000330,-0.075511,0.085885,-3.711237,-0.024742,0.161396,...,4.145141e-07,0.000001,0.000029,1.081643e-06,1.543797e-06,0.000010,0.000009,0.000017,0.000007,0
2,2,-0.000237,0.000155,0.000155,0.000325,-0.086746,0.085885,-4.165139,-0.027768,0.172631,...,7.530225e-07,0.000001,0.000029,9.076508e-07,1.637235e-06,0.000010,0.000009,0.000017,0.000007,0
3,2,-0.000318,0.000201,0.000201,0.000324,-0.086746,0.085885,-5.577939,-0.037186,0.172631,...,8.024451e-07,0.000002,0.000029,7.330045e-07,1.691937e-06,0.000011,0.000008,0.000017,0.000006,0
4,2,-0.000366,0.000222,0.000222,0.000321,-0.086746,0.085885,-6.430886,-0.042873,0.172631,...,2.809569e-06,0.000002,0.000028,5.764813e-07,1.662168e-06,0.000011,0.000008,0.000017,0.000005,0
5,2,-0.000416,0.000205,0.000205,0.000316,-0.091967,0.085885,-7.301272,-0.048675,0.177852,...,5.603962e-06,0.000003,0.000028,4.305069e-07,1.532440e-06,0.000011,0.000008,0.000016,0.000005,0
6,2,-0.000482,0.000165,0.000165,0.000314,-0.091967,0.085885,-8.465204,-0.056435,0.177852,...,5.973719e-06,0.000003,0.000028,2.897140e-07,1.389275e-06,0.000011,0.000009,0.000015,0.000006,0
7,2,-0.000476,0.000117,0.000117,0.000313,-0.091967,0.085885,-8.354190,-0.055695,0.177852,...,6.365769e-06,0.000004,0.000028,2.363532e-07,1.210596e-06,0.000010,0.000009,0.000014,0.000006,0
8,2,-0.000451,0.000114,0.000114,0.000307,-0.091967,0.085885,-7.923766,-0.052825,0.177852,...,7.925245e-06,0.000004,0.000027,2.201624e-07,1.020257e-06,0.000010,0.000009,0.000013,0.000006,0
9,2,-0.000460,0.000106,0.000106,0.000301,-0.091967,0.096904,-8.070069,-0.053800,0.188871,...,7.930224e-06,0.000004,0.000026,2.068819e-07,8.366389e-07,0.000010,0.000008,0.000013,0.000006,0


In [16]:
print('ALL_accuracy = ', )
ALL_accuracy = classifier(dataset=ALL, model=mod)
ALL_accuracy 

ALL_accuracy = 
window_size =  3.0  sec
Model- 1  -  f1 score:  0.7435409075852932
Model- 2  -  f1 score:  0.7614341364882412
Model- 3  -  f1 score:  0.6018181818181818


0.7022644086305722

In [17]:
d ={'ALL': [ALL_accuracy]}

In [18]:
df_accuracy = pd.DataFrame(data=d)

In [19]:
df_accuracy

,ALL
0,0.702264
